In [47]:
import tensorflow as tf
import numpy as np

# Last ned og last Fashion-MNIST-datasettet
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [49]:
# Normaliser bildedataene til å være mellom 0 og 1
x_train = x_train / 255.0
x_test = x_test / 255.0

In [51]:
# Flatt ut bildene
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

In [53]:
x_val = x_train[:10000]
partial_x_train = x_train[10000:]
y_val = y_train[:10000]
partial_y_train = y_train[10000:]

In [55]:
## import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras import regularizers
from tensorflow.keras.regularizers import l2
import datetime
import numpy as np

In [57]:

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Define modell
model = Sequential([
    Dense(512, activation='relu', kernel_regularizer=l2(0.001), input_shape=(784,), kernel_initializer=HeNormal()),
    BatchNormalization(),
    Dropout(0.1),
    Dense(128, activation='relu', kernel_regularizer=l2(0.001), kernel_initializer=HeNormal()),
    BatchNormalization(),
    Dropout(0.1),
    Dense(64, activation='relu', kernel_regularizer=l2(0.001), kernel_initializer=HeNormal()),
    BatchNormalization(),
    Dropout(0.1),
    Dense(10, activation='softmax', kernel_initializer=HeNormal()),
])

# Early stopping
def run_experiment_with_early_stopping(model, loss_recording_list, lr=0.0001, x_train=None, y_train=None, val_x=None, val_y=None, epochs=100, patience=5):
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=optimizer, loss=SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])
    
    # Early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)
    
    # Train the model
    history = model.fit(
        x_train,
        y_train,
        epochs=epochs,
        batch_size=184,
        validation_data=(val_x, val_y),
        callbacks=[early_stopping, tensorboard_callback]
    )
    
    # Record the losses
    loss_recording_list.append({
        'train_loss': history.history['loss'],
        'val_loss': history.history['val_loss']
    })
    
    # Evaluate the model on the test set
    test_loss, test_accuracy = model.evaluate(x_test, y_test)
    print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")
    
    # Get predictions
    predictions = model.predict(x_test)
    predictions = predictions.argmax(axis=1)
    
    # Evaluate predictions
    evaluate(y_test, predictions)

# Dummy evaluate function for demonstration
def evaluate(y_true, y_pred):
    accuracy = np.mean(y_true == y_pred)
    print(f"Evaluation Accuracy: {accuracy}")

# List to record losses
loss_recording_list = []

# Run the experiment
run_experiment_with_early_stopping_keras(
    model=model,
    loss_recording_list=loss_recording_list,
    lr=0.0001,
    x_train=x_train,
    y_train=y_train,
    val_x=x_val,
    val_y=y_val,
    epochs=100,
    patience=5
)

Epoch 1/100
327/327 ━━━━━━━━━━━━━━━━━━━━ 15s 25ms/step - accuracy: 0.6174 - loss: 2.5839 - val_accuracy: 0.8333 - val_loss: 1.8595
Epoch 2/100
327/327 ━━━━━━━━━━━━━━━━━━━━ 8s 23ms/step - accuracy: 0.8266 - loss: 1.8210 - val_accuracy: 0.8675 - val_loss: 1.6391
Epoch 3/100
327/327 ━━━━━━━━━━━━━━━━━━━━ 9s 26ms/step - accuracy: 0.8441 - loss: 1.6705 - val_accuracy: 0.8777 - val_loss: 1.5192
Epoch 4/100
327/327 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step - accuracy: 0.8624 - loss: 1.5507 - val_accuracy: 0.8881 - val_loss: 1.4160
Epoch 5/100
327/327 ━━━━━━━━━━━━━━━━━━━━ 7s 22ms/step - accuracy: 0.8696 - loss: 1.4469 - val_accuracy: 0.8915 - val_loss: 1.3321
Epoch 6/100
327/327 ━━━━━━━━━━━━━━━━━━━━ 8s 23ms/step - accuracy: 0.8782 - loss: 1.3611 - val_accuracy: 0.8987 - val_loss: 1.2501
Epoch 7/100
327/327 ━━━━━━━━━━━━━━━━━━━━ 8s 23ms/step - accuracy: 0.8816 - loss: 1.2844 - val_accuracy: 0.8946 - val_loss: 1.2019
Epoch 8/100
327/327 ━━━━━━━━━━━━━━━━━━━━ 7s 22ms/step - accuracy: 0.8854 - loss: 1.2186 -